In [4]:
import pandas as pd
from sklearn.cluster import AffinityPropagation
from dotenv import load_dotenv
import os
import warnings
warnings.filterwarnings("ignore")

load_dotenv()
directory = os.getenv("path")

In [58]:
# NOTE: The save csv the values are the indices of the stations in the original dataset that are exmplars
# NOTE: starting from 0 to n-1 where n is the number of stations in the dataset.

model = AffinityPropagation()

# Clustering with AP.
def clustering(dir, output_dir, model, dataset):
    paths = os.listdir(dir)
    files = sorted([file for file in paths if file.endswith(".csv")])

    ap_res = {}
    for file in files:
        df = pd.read_csv(os.path.join(dir, file))

        if "Unnamed: 0" in df.columns:
            df.drop(df.columns[0], axis=1, inplace=True)
            df.set_index(df.columns[0], inplace=True)

        else:
            df.set_index(df.columns[0], inplace=True)

        df = df.fillna(df.median())

        # AP clustering.
        model.fit(df)
        stations_index = list(model.cluster_centers_indices_)


        indices= {}
        for index, station in enumerate(list(stations_index)):
            indices[index] = station

        # assingning cluster labels to the stations
        labels = model.labels_
        ap_labels = [indices[station] for station in labels]

        ap_res[file.split("/")[-1].split(".")[0]] = list(ap_labels)
        result = pd.DataFrame(ap_res).set_index(df.index)
    

    try:
        result.to_csv(os.path.join(output_dir, f'{dataset}.csv'), index=True)
        print(f'<-- clustering done -->')
    except Exception as e:
        print(e)


    return result

# NOTE: Clustering with AP.
datasets = ["func", "func_climate", "all_attributes"]

for dataset in datasets:
    dir = os.path.join(directory, "computed_data/pca_results", dataset)
    outputdir = os.path.join(directory, "computed_data/ap_results")
    res = clustering(dir, outputdir, model, dataset)

<-- clustering done -->
<-- clustering done -->
<-- clustering done -->


In [ ]:
# check the number of clusters
dir = os.path.join(directory, 'computed_data/ap_results')
paths = os.listdir(dir)
files = sorted([file for file in paths if file.endswith(".csv")])

ap_results = {}
for file in files:
    df = pd.read_csv(os.path.join(dir, file))
    for col in df.columns:
        if col == 'station_id':
            continue
        ap_results[col] = df[col].nunique()
